In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
import sys
sys.path.append("/media/DATABRUT/DB_DDC/serverGPU/Cache_GPU_Ai/fiber_matcher/FFMatcher/training_match/")

from configs import *
from FM_match import *
from helper import *
import pickle
import shutil

set_seed(random_seed)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# change the directory to save the outputs
from run_match_input import *

# Create the directories
images_dir = f"{dir_save_output}/images" #dir of images used in training
dir_tempo = f"{dir_save_output}/tempo/" #dir to dump the temporary files
dir_embedding = f"{dir_save_output}/VAE_embed"
dir_save_npz = f"{dir_save_output}/npz_256"
dir_save_prediction_output = f"{dir_save_output}/prediction_output"
dir_save_cellpose_masks = f"{dir_save_output}/out_CP_masks"

for d in [dir_save_output, images_dir, dir_tempo, dir_embedding, dir_save_npz, dir_save_prediction_output, dir_save_cellpose_masks]:
    os.makedirs(d, exist_ok=True)

# IHF or not
IHF1 = param_img1[0]=="fluorescence"
IHF2 = param_img2[0]=="fluorescence"

img1 = "22-082_10X_DAPI_LAM_DYS_COL4_1-Scene-3-TAG03"
img2 = "22-082_10X_NADH_9-Scene-3-TAG03"

Processing 22-082_10X_DAPI_LAM_DYS_COL4_1-Scene-3-TAG03 and 22-082_10X_NADH_9-Scene-3-TAG03


# Save step prediction

In [2]:
dir_prediction_single_pair = f"{dir_save_prediction_output}/{img1}___vs___{img2}"

with open(f"{dir_prediction_single_pair}/step_prediction.pkl", "rb") as f:
    step_prediction = pickle.load(f)

masks1, _, _ = segment_image(
    img_path=f"{images_dir}/{img1}.png",  
    CP_model_name=CP_model_name_1,
    savedir=dir_save_cellpose_masks
)

masks2, _, _ = segment_image(
    img_path=f"{images_dir}/{img2}.png",  
    CP_model_name=CP_model_name_2,
    savedir=dir_save_cellpose_masks
)

In [5]:
imgs = [img1, img2]
masks = [masks1, masks2]
IHFs = [IHF1, IHF2]

# Start prediction
for img_index in range(2):
    labels_step = []
    list_roi_plot = [l[img_index] for l in labels_step]
    draw_instructions = get_list_params_contours(list_roi_plot, masks[img_index], IHFs[img_index], "on-going")
    save_image_per_step(
        imgs[img_index], images_dir, 
        draw_instructions,
        dir_prediction_single_pair,
        suffix="Step0",
        dpi=50,
    )

# 1_initial_guess
for img_index in range(2):
    labels_step = step_prediction['1_initial_guess']
    list_roi_plot = [l[img_index] for l in labels_step]
    draw_instructions = get_list_params_contours(list_roi_plot, masks[img_index], IHFs[img_index], "on-going")
    save_image_per_step(
        imgs[img_index], images_dir, 
        draw_instructions,
        dir_prediction_single_pair,
        suffix="Step1",
        dpi=50,
    )

# 2_selected_combs_from_initial_guess
for img_index in range(2):
    labels_step = step_prediction['2_selected_combs_from_initial_guess']
    list_roi_plot = [l[img_index] for l in labels_step]
    draw_instructions = get_list_params_contours(list_roi_plot, masks[img_index], IHFs[img_index], "confirmed")
    save_image_per_step(
        imgs[img_index], images_dir, 
        draw_instructions,
        dir_prediction_single_pair,
        suffix="Step2",
        dpi=50,
    )

# 3_local_prediction
for img_index in range(2):
    for i,labels_step in enumerate(step_prediction['3_local_prediction']):
        list_roi_plot = [l[img_index] for l in labels_step]
        draw_instructions = get_list_params_contours(list_roi_plot, masks[img_index], IHFs[img_index], "confirmed")
        save_image_per_step(
            imgs[img_index], images_dir, 
            draw_instructions,
            dir_prediction_single_pair,
            suffix=f"Step3-{i}",
            dpi=50,
        )

# 4_unannotated_prediction
for img_index in range(2):
    labels_step = step_prediction['4_unannotated_prediction'][-1]
    list_roi_plot = [l[img_index] for l in labels_step]
    draw_instructions = get_list_params_contours(list_roi_plot, masks[img_index], IHFs[img_index], "confirmed")
    save_image_per_step(
        imgs[img_index], images_dir, 
        draw_instructions,
        dir_prediction_single_pair,
        suffix=f"Step4",
        dpi=50,
    )

# Make time-step video for prediction

In [17]:
left_images, right_images = [], []
suffixes = ["Step0", "Step1", "Step2"]
for i in range(len(step_prediction['3_local_prediction'])):
    suffixes.append(f"Step3-{i}")
suffixes.append("Step4")

for suffix in suffixes:
    left_images.append(f"{dir_prediction_single_pair}/{img1}_{suffix}.png")
    right_images.append(f"{dir_prediction_single_pair}/{img2}_{suffix}.png")

In [18]:
import cv2
import numpy as np
from PIL import Image
from pathlib import Path

def _load_rgb(path):
    # PIL avoids cv2’s BGR gotcha
    return np.array(Image.open(path).convert("RGB"))

def _resize_keep_aspect(img, target_h=None, target_w=None):
    h, w = img.shape[:2]
    if target_h is not None:
        scale = target_h / h
    elif target_w is not None:
        scale = target_w / w
    else:
        return img
    new_w, new_h = int(round(w*scale)), int(round(h*scale))
    return np.array(Image.fromarray(img).resize((new_w, new_h), Image.BILINEAR))

def _pad_to_same_height(img_left, img_right, pad_color=(0,0,0)):
    hL, wL = img_left.shape[:2]
    hR, wR = img_right.shape[:2]
    H = max(hL, hR)
    if hL < H:
        pad = np.full((H - hL, wL, 3), pad_color, dtype=np.uint8)
        img_left = np.vstack([img_left, pad])
    if hR < H:
        pad = np.full((H - hR, wR, 3), pad_color, dtype=np.uint8)
        img_right = np.vstack([img_right, pad])
    return img_left, img_right

def _annotate(img, text, color, font_scale=1, thickness=2, margin=6):
    overlay = img.copy()
    cv2.putText(
        overlay, text, (margin, margin + 24),
        cv2.FONT_HERSHEY_SIMPLEX, font_scale, color, thickness, cv2.LINE_AA
    )
    return overlay

def make_side_by_side_video(
    pairs,                     # list of (path_left, path_right)
    out_path="out.mp4",
    fps=10,
    target_height=720,         # final frame height
    add_filenames=True,
    left_title="Laminin",           # e.g. "Slide A"
    right_title="NADH",          # e.g. "Slide B"
    fourcc="mp4v",             # "mp4v" (broad) or "avc1" (H.264 if available)
):
    pairs = [(Path(a), Path(b)) for a,b in pairs]
    assert len(pairs) > 0, "No input pairs."

    frames = []
    for a, b in pairs:
        L = _load_rgb(a)
        R = _load_rgb(b)

        # 1) scale both to the same target height
        L = _resize_keep_aspect(L, target_h=target_height)
        R = _resize_keep_aspect(R, target_h=target_height)

        # 2) pad to identical height (if rounding created a 1px diff)
        L, R = _pad_to_same_height(L, R)

        # 3) optional labels (titles override filenames if given)
        if add_filenames:
            txtL = left_title if left_title is not None else a.name
            txtR = right_title if right_title is not None else b.name
            L = _annotate(L, txtL, (255,255,255))
            R = _annotate(R, txtR, (0,0,0))

        # 4) concatenate horizontally
        frame = np.hstack([L, R]).astype(np.uint8)
        frames.append(frame)

    H, W = frames[0].shape[:2]
    writer = cv2.VideoWriter(
        str(out_path),
        cv2.VideoWriter_fourcc(*fourcc),
        fps,
        (W, H)
    )

    for f in frames:
        # OpenCV expects BGR
        writer.write(cv2.cvtColor(f, cv2.COLOR_RGB2BGR))
    writer.release()
    print(f"Saved {len(frames)} frames → {out_path} ({W}x{H}@{fps}fps)")

pairs = list(zip(left_images, right_images))
make_side_by_side_video(
    pairs, "comparison.mp4", fps=2, target_height=800,
    left_title="Laminin", right_title="NADH"
)

Saved 15 frames → comparison.mp4 (1565x800@2fps)
